In [10]:
import pandas as pd
import yfinance as yf
import numpy as np
import pmdarima as pm
from datetime import datetime, timedelta

# Function to download data for a list of tickers
def download_and_adjust_data(tickers, start_date, end_date):
    dataframes = {}
    for ticker in tickers:
        data = yf.download(ticker, start=start_date, end=end_date)
        data["Adjustment Multiplier"] = data["Adj Close"] / data["Close"]
        data["Adj Open"] = data["Open"] * data["Adjustment Multiplier"]
        data["returns"] = ((data["Adj Open"] - data["Adj Close"].shift(1)) / data["Adj Close"].shift(1)).fillna(0)
        dataframes[ticker] = data
    return dataframes

# Function to merge and prepare returns
def prepare_merged_data(dataframes, backtest=True):
    Merged = pd.concat([dataframes[ticker].add_prefix(f"{ticker}_") for ticker in dataframes], axis=1).dropna()
    returns_columns = [f"{ticker}_returns" for ticker in dataframes]

    if "SPY" in dataframes:
        returns_columns.append("SPY_tplus1")
        Merged["SPY_tplus1"] = Merged["SPY_returns"].shift(-1).dropna()


    Merged_Returns = Merged[returns_columns].copy().dropna()
    Merged_Returns = round(Merged_Returns * 100, 2)
    if backtest:
        return Merged_Returns
    else:
        return Merged_Returns.tail(30)


# Function to train the ARIMAX model
def train_arimax_model(X, Y):
    model = pm.arima.auto_arima(y=Y, X=X)
    return model

def make_prediction(model, tickers, production_start_date, production_end_date):
    production_dataframes = download_and_adjust_data(tickers, production_start_date, production_end_date)
    
    Production_Merged = pd.concat(
        [production_dataframes[ticker].add_prefix(f"{ticker}_") for ticker in production_dataframes],
        axis=1
    ).dropna()

    # Ensure all necessary returns columns are included
    returns_columns = [f"{ticker}_returns" for ticker in production_dataframes]
    Production_Merged_Returns = Production_Merged[returns_columns].copy().dropna()
    Production_Shifted_Merged_Returns = round(Production_Merged_Returns.tail(1) * 100, 2)
    SP500_Production_Dataset = Production_Shifted_Merged_Returns.copy()

    # Ensure the shape matches the model's expected exogenous input shape
    prediction = model.predict(X=SP500_Production_Dataset, n_periods=1)
    next_day = (SP500_Production_Dataset.index[0] + timedelta(days=1)).strftime("%Y-%m-%d")
    print(f"The model predicts the S&P 500 to return {prediction[0]:.2f}% on the open of {next_day}")

In [13]:
tickers_ls = ['SPY', 'XLB', 'XLE','XLF', 'XLI', 'XLK', 'XLP', 'XLU', 'XLV', 'XLY', 'XLRE', 'XLC' ]
df_dict = download_and_adjust_data(tickers_ls, start_date='2010-01-01', end_date='2024-11-16')
merged_returns_df = prepare_merged_data(df_dict, backtest=True)

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


In [16]:
X = merged_returns_df.drop("SPY_tplus1", axis=1)
y = merged_returns_df["SPY_tplus1"]
y

Date
2018-06-19    0.28
2018-06-20   -0.00
2018-06-21    0.52
2018-06-22   -0.47
2018-06-25    0.24
              ... 
2024-11-08    0.27
2024-11-11   -0.01
2024-11-12    0.08
2024-11-13    0.02
2024-11-14   -0.61
Name: SPY_tplus1, Length: 1614, dtype: float64

In [26]:
from tqdm import tqdm 

window_size = 30
predictions = []  
dates = y.index[window_size:]  

for end_idx in tqdm(range(window_size, len(merged_returns_df)), desc="Iterating through penis:"):
        # Define the training window
        start_idx = end_idx - window_size
        y_train = y.iloc[start_idx:end_idx]
        X_train = X.iloc[start_idx:end_idx]
        model = train_arimax_model(X_train, y_train)
        pred_features = pd.DataFrame(X.iloc[end_idx]).T
        prediction = model.predict(X=pred_features, n_periods= 1)
        predictions.append(prediction.iloc[0])

        print(f"The model predicts the S&P 500 to return {prediction.iloc[0]:.2f}% for t+1 from {end_idx}")

results = pd.DataFrame({'Date': dates, 'Prediction': predictions})
results.set_index('Date', inplace=True)
results.to_csv(f'V2_rolling_window_predictions_{window_size}.csv')

Iterating through penis::   0%|          | 0/1584 [00:00<?, ?it/s]f:\Anaconda\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:836: ValueWarning: No supported index is available. Prediction results will be given with an integer index beginning at `start`.
  return get_prediction_index(
f:\Anaconda\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:836: FutureWarning: No supported index is available. In the next version, calling this method in a model without a supported index will result in an exception.
  return get_prediction_index(
Iterating through penis::   0%|          | 1/1584 [00:02<56:32,  2.14s/it]

The model predicts the S&P 500 to return -0.35% for t+1 from 30


f:\Anaconda\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:836: ValueWarning: No supported index is available. Prediction results will be given with an integer index beginning at `start`.
  return get_prediction_index(
f:\Anaconda\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:836: FutureWarning: No supported index is available. In the next version, calling this method in a model without a supported index will result in an exception.
  return get_prediction_index(
Iterating through penis::   0%|          | 2/1584 [00:06<1:33:44,  3.56s/it]

The model predicts the S&P 500 to return 0.67% for t+1 from 31


f:\Anaconda\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:836: ValueWarning: No supported index is available. Prediction results will be given with an integer index beginning at `start`.
  return get_prediction_index(
f:\Anaconda\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:836: FutureWarning: No supported index is available. In the next version, calling this method in a model without a supported index will result in an exception.
  return get_prediction_index(
Iterating through penis::   0%|          | 3/1584 [00:11<1:54:49,  4.36s/it]

...

The model predicts the S&P 500 to return 1.83% for t+1 from 1611


Iterating through penis:: 100%|█████████▉| 1583/1584 [1:50:39<00:03,  3.81s/it]

The model predicts the S&P 500 to return -0.47% for t+1 from 1612


Iterating through penis:: 100%|██████████| 1584/1584 [1:50:45<00:00,  4.20s/it]

The model predicts the S&P 500 to return 0.33% for t+1 from 1613
